# Train Model

In [218]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("ucl_match_data.csv")
df.columns

Index(['Date', 'Venue', 'Result', 'GF', 'GA', 'xG', 'xGA', 'Poss', 'Sh', 'SoT',
       'Dist', 'FK', 'PK', 'PKatt', 'Team ID', 'Opponent ID', 'Points',
       'Goal Difference', 'xG Difference', 'Shot Accuracy'],
      dtype='object')

In [219]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

features = ['GF', 'GA', 'xG', 'xGA', 'Sh', 'SoT', 'Goal Difference', 'xG Difference', 'Shot Accuracy', 'Poss', 'FK', 'PK', 'PKatt']

target = 'Result'

# Split data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the model: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy of the model: 1.0000

Classification Report:
              precision    recall  f1-score   support

           D       1.00      1.00      1.00        10
           L       1.00      1.00      1.00        25
           W       1.00      1.00      1.00        23

    accuracy                           1.00        58
   macro avg       1.00      1.00      1.00        58
weighted avg       1.00      1.00      1.00        58



In [220]:
def handle_draw_with_penalties(prediction):
    if prediction == 'D':
        pen_result = np.random.choice(['W', 'L'])
        print("After Penalties: " + pen_result)
        return pen_result
    else:
        return prediction

# Aggregate previous matches statistics for each team
def aggregate_team_stats(team_id, df):
    # Filter matches for the team
    team_matches = df[(df['Team ID'] == team_id)]

    aggregated_stats = team_matches[features].mean()  # Take the mean of each feature
    return aggregated_stats

def predict_match_result(team_id_1, team_id_2):
    # Get aggregated stats for both teams
    team_1_data = aggregate_team_stats(team_id_1, df)
    team_2_data = aggregate_team_stats(team_id_2, df)

    # Combine features for the prediction (Goal differences, shot accuracy, etc.)
    input_features = {
        'GF': team_1_data['GF'],  # Goals scored by team 1
        'GA': team_2_data['GF'],  # Goals scored by team 2
        'xG': team_1_data['xG'],  # Expected goals scored by team 1
        'xGA': team_2_data['xG'], # Expected goals scored by team 2
        'Sh': team_1_data['Sh'],
        'SoT': team_1_data['SoT'],  # Shots on target by team 1
        'Goal Difference': team_1_data['GF'] - team_2_data['GF'],  # Difference in goals
        'xG Difference': team_1_data['xG'] - team_2_data['xG'],  # Difference in expected goals
        'Shot Accuracy': team_1_data['SoT'] / team_1_data['Sh'],  # Shot accuracy of team 1
        'Poss': team_1_data['Poss'],  # Possession stats
        'FK': team_1_data['FK'],
        'PK': team_1_data['PK'],
        'PKatt': team_1_data['PKatt']
    }

    # Convert input features into a DataFrame for prediction
    input_df = pd.DataFrame([input_features])

    # Predict match outcome
    predicted_outcome = model.predict(input_df)[0]

    return predicted_outcome

In [221]:
team_1_id = 1  # Barcelona FC
team_2_id = 0  # Liverpool FC

match_result = predict_match_result(team_1_id, team_2_id)

print(f"Predicted match result: {match_result}")

match_result = handle_draw_with_penalties(match_result)

Predicted match result: W


# Tournament Prediction

### Team IDs

In [222]:
team_map = {
    0: 'Liverpool',
    1: 'Barcelona',
    2: 'Arsenal',
    3: 'Internazionale',
    4: 'Atletico Madrid',
    5: 'Bayer Leverkusen',
    6: 'Lille',
    7: 'Aston Villa',
    8: 'Atalanta',
    9: 'Dortmund',
    10: 'Real Madrid',
    11: 'Bayern Munich',
    12: 'Milan',
    13: 'PSV Eindhoven',
    14: 'Paris Saint-Germain',
    15: 'Benfica',
    16: 'Monaco',
    17: 'Brest',
    18: 'Feyenoord',
    19: 'Juventus',
    20: 'Celtic',
    21: 'Manchester City',
    22: 'Sporting CP',
    23: 'Club Brugge',
    24: 'Dinamo Zagreb',
    25: 'Stuttgart',
    26: 'Shakhtar Donetsk',
    27: 'Bologna',
    28: 'Red Star Belgrade',
    29: 'Sturm Graz',
    30: 'Sparta Prague',
    31: 'RB Leipzig',
    32: 'Girona',
    33: 'Red Bull Salzburg',
    34: 'Slovan Bratislava',
    35: 'Young Boys'
}

# Function to map team ID to team name
def get_team_name(team_id):
    return team_map.get(team_id, 'Unknown Team')

def get_team_id(team_name):
    for team_id, name in team_map.items():
        if name == team_name:
            return team_id
    return None

## Simulate Tournament

### Playoffs

In [223]:
playoffs = {
    1: ("Paris Saint-Germain", "Brest"),
    2: ("Atalanta", "Club Brugge"),
    3: ("Real Madrid", "Manchester City"),
    4: ("Juventus", "PSV Eindhoven"),
    5: ("Benfica", "Monaco"),
    6: ("Sporting CP", "Dortmund"),
    7: ("Bayern Munich", "Celtic"),
    8: ("Milan", "Feyenoord"),
}

# Simulate the playoffs round and store winners
winners_playoffs = {}
print("Simulating Playoffs:\n")
for match_id, teams in playoffs.items():
    print(f"Playoff {match_id}: {teams[0]} vs {teams[1]}")

    result = predict_match_result(get_team_id(teams[0]), get_team_id(teams[1]))

    print_result = "Winner of Playoff " + str(match_id) + ": "
    if result == "D":
        winner = np.random.choice([teams[0], teams[1]])
        print_result += str(winner) + " (after penalties)"

    elif result == "W":
        winner = teams[0]
        print_result += (winner)

    elif result == "L":
        winner = teams[1]
        print_result += (winner)

    winners_playoffs[match_id] = winner
    print(print_result + "\n")



Simulating Playoffs:

Playoff 1: Paris Saint-Germain vs Brest
Winner of Playoff 1: Paris Saint-Germain (after penalties)

Playoff 2: Atalanta vs Club Brugge
Winner of Playoff 2: Atalanta

Playoff 3: Real Madrid vs Manchester City
Winner of Playoff 3: Real Madrid (after penalties)

Playoff 4: Juventus vs PSV Eindhoven
Winner of Playoff 4: PSV Eindhoven

Playoff 5: Benfica vs Monaco
Winner of Playoff 5: Monaco (after penalties)

Playoff 6: Sporting CP vs Dortmund
Winner of Playoff 6: Dortmund

Playoff 7: Bayern Munich vs Celtic
Winner of Playoff 7: Bayern Munich

Playoff 8: Milan vs Feyenoord
Winner of Playoff 8: Feyenoord



### Round of 16

In [224]:
round_of_16 = {
    1: ("Liverpool", winners_playoffs[1]),
    2: ("Barcelona", winners_playoffs[5]),
    3: ("Arsenal", winners_playoffs[4]),
    4: ("Internazionale", winners_playoffs[8]),
    5: ("Atletico Madrid", winners_playoffs[3]),
    6: ("Bayer Leverkusen", winners_playoffs[7]),
    7: ("Lille", winners_playoffs[2]),
    8: ("Aston Villa", winners_playoffs[6]),
}

# Simulate the round of 16 and store winners
winners_rnd_16 = {}
print("Simulating Round of 16:\n")
for match_id, teams in round_of_16.items():
    print(f"R16 {match_id}: {teams[0]} vs {teams[1]}")

    result = predict_match_result(get_team_id(teams[0]), get_team_id(teams[1]))

    print_result = "Winner of R16 " + str(match_id) + ": "

    if result == "D":
        winner = np.random.choice([teams[0], teams[1]])
        print_result += str(winner) + " (after penalties)"

    elif result == "W":
        winner = teams[0]
        print_result += (winner)

    elif result == "L":
        winner = teams[1]
        print_result += (winner)

    winners_rnd_16[match_id] = winner
    print(print_result + "\n")




Simulating Round of 16:

R16 1: Liverpool vs Paris Saint-Germain
Winner of R16 1: Liverpool (after penalties)

R16 2: Barcelona vs Monaco
Winner of R16 2: Barcelona

R16 3: Arsenal vs PSV Eindhoven
Winner of R16 3: Arsenal (after penalties)

R16 4: Internazionale vs Feyenoord
Winner of R16 4: Feyenoord

R16 5: Atletico Madrid vs Real Madrid
Winner of R16 5: Atletico Madrid (after penalties)

R16 6: Bayer Leverkusen vs Bayern Munich
Winner of R16 6: Bayern Munich

R16 7: Lille vs Atalanta
Winner of R16 7: Lille (after penalties)

R16 8: Aston Villa vs Dortmund
Winner of R16 8: Dortmund



### Quarter Finals

In [225]:
quarter_finals = {
    1: (winners_rnd_16[1], winners_rnd_16[7]),
    2: (winners_rnd_16[2], winners_rnd_16[8]),
    3: (winners_rnd_16[5], winners_rnd_16[3]),
    4: (winners_rnd_16[6], winners_rnd_16[4]),
}

# Simulate the quarter finals and store winners
winner_quarter = {}
print("Simulating Quarter Finals:\n")
for match_id, teams in quarter_finals.items():
    print(f"QF {match_id}: {teams[0]} vs {teams[1]}")

    result = predict_match_result(get_team_id(teams[0]), get_team_id(teams[1]))

    print_result = "Winner of QF " + str(match_id) + ": "

    if result == "D":
        winner = np.random.choice([teams[0], teams[1]])
        print_result += str(winner) + " (after penalties)"

    elif result == "W":
        winner = teams[0]
        print_result += (winner)

    elif result == "L":
        winner = teams[1]
        print_result += (winner)

    winner_quarter[match_id] = winner
    print(print_result + "\n")



Simulating Quarter Finals:

QF 1: Liverpool vs Lille
Winner of QF 1: Liverpool (after penalties)

QF 2: Barcelona vs Dortmund
Winner of QF 2: Barcelona

QF 3: Atletico Madrid vs Arsenal
Winner of QF 3: Arsenal (after penalties)

QF 4: Bayern Munich vs Feyenoord
Winner of QF 4: Bayern Munich (after penalties)



### Semi Final

In [226]:
semi_finals = {
    1: (winner_quarter[1], winner_quarter[3]),
    2: (winner_quarter[2], winner_quarter[4]),
}

# Simulate the semi finals and store winners
winner_semis = {}
print("Simulating Semi Finals:\n")
for match_id, teams in semi_finals.items():
    print(f"SF {match_id}: {teams[0]} vs {teams[1]}")

    result = predict_match_result(get_team_id(teams[0]), get_team_id(teams[1]))

    print_result = "Winner of SF " + str(match_id) + ": "

    if result == "D":
        winner = np.random.choice([teams[0], teams[1]])
        print_result += str(winner) + " (after penalties)"

    elif result == "W":
        winner = teams[0]
        print_result += (winner)

    elif result == "L":
        winner = teams[1]
        print_result += (winner)

    winner_semis[match_id] = winner
    print(print_result + "\n")



Simulating Semi Finals:

SF 1: Liverpool vs Arsenal
Winner of SF 1: Liverpool (after penalties)

SF 2: Barcelona vs Bayern Munich
Winner of SF 2: Barcelona



### Final

In [227]:
final = {
    1: (winner_semis[1], winner_semis[2])
}

# Simulate the final
print("Simulating UCL Final:\n")
for match_id, teams in final.items():
    print(f"Final: {teams[0]} vs {teams[1]}")

    result = predict_match_result(get_team_id(teams[0]), get_team_id(teams[1]))

    print_result = "Champions League Winner: "

    if result == "D":
        winner = np.random.choice([teams[0], teams[1]])
        print_result += str(winner) + " (after penalties)"

    elif result == "W":
        winner = teams[0]
        print_result += (winner)

    elif result == "L":
        winner = teams[1]
        print_result += (winner)
    final_winner = winner
    print(print_result + "\n")
    print("Congratulations to " + winner + "!")

Simulating UCL Final:

Final: Liverpool vs Barcelona
Champions League Winner: Barcelona

Congratulations to Barcelona!


## Display Result

In [228]:
def print_bracket():
    print("Champions League Tournament:\n")

    # Display Playoffs
    print("Playoffs:")
    for i, match in playoffs.items():
        print(f"Playoff {i}: {match[0]} vs {match[1]} -> Winner: {winners_playoffs[i]}")

    # Display Round of 16
    print("\nRound of 16:")
    for i, match in round_of_16.items():
        print(f"Round {i}: {match[0]} vs {match[1]} -> Winner: {winners_rnd_16[i]}")

    # Display Quarterfinals
    print("\nQuarterfinals:")
    for i, match in quarter_finals.items():
        print(f"Quarterfinal {i}: {match[0]} vs {match[1]} -> Winner: {winner_quarter[i]}")

    # Display Semifinals
    print("\nSemifinals:")
    for i, match in semi_finals.items():
        print(f"Semifinal {i}: {match[0]} vs {match[1]} -> Winner: {winner_semis[i]}")

    # Display Final
    print("\nFinal:")
    match = final[1]
    print(f"Final: {match[0]} vs {match[1]} -> Winner: {final_winner}")
    print("\nTournament Complete!")

# Run the printing function
print_bracket()


Champions League Tournament:

Playoffs:
Playoff 1: Paris Saint-Germain vs Brest -> Winner: Paris Saint-Germain
Playoff 2: Atalanta vs Club Brugge -> Winner: Atalanta
Playoff 3: Real Madrid vs Manchester City -> Winner: Real Madrid
Playoff 4: Juventus vs PSV Eindhoven -> Winner: PSV Eindhoven
Playoff 5: Benfica vs Monaco -> Winner: Monaco
Playoff 6: Sporting CP vs Dortmund -> Winner: Dortmund
Playoff 7: Bayern Munich vs Celtic -> Winner: Bayern Munich
Playoff 8: Milan vs Feyenoord -> Winner: Feyenoord

Round of 16:
Round 1: Liverpool vs Paris Saint-Germain -> Winner: Liverpool
Round 2: Barcelona vs Monaco -> Winner: Barcelona
Round 3: Arsenal vs PSV Eindhoven -> Winner: Arsenal
Round 4: Internazionale vs Feyenoord -> Winner: Feyenoord
Round 5: Atletico Madrid vs Real Madrid -> Winner: Atletico Madrid
Round 6: Bayer Leverkusen vs Bayern Munich -> Winner: Bayern Munich
Round 7: Lille vs Atalanta -> Winner: Lille
Round 8: Aston Villa vs Dortmund -> Winner: Dortmund

Quarterfinals:
Quarterf